In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [ ]:
data.head()

In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# **Preprocessing the data**

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
ordinal_encoder = OrdinalEncoder()
std_scaler = StandardScaler()
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
def change(x):
    return ord(x) if x != 'nan' else 1

def preprocess_data(X):
    #Transforming `PassengerId`
    X['PassengerId'] = X['PassengerId'].str.split('_').str[0]
    
    #Imputing missing values
    freq_col = [
        'HomePlanet', 'Destination', 
        'CryoSleep', 'VIP'
    ]
    mean_col = [
        'Age', 'RoomService', 'FoodCourt', 
        'ShoppingMall', 'Spa', 'VRDeck'
    ]
    
    for tmp_freq in freq_col:
        X[tmp_freq] = imp_freq.fit_transform(
            X[tmp_freq].values.reshape(-1, 1)
        )
    
    for tmp_mean in mean_col:
        X[tmp_mean] = imp_mean.fit_transform(
            X[tmp_mean].values.reshape(-1, 1)
        )
    
    #Ordinal Encoding the selected columns
    ordinal_enc_columns = [
        'HomePlanet', 'CryoSleep', 
        'Destination', 'VIP',
        'PassengerId'
    ]
    for tmp_col in ordinal_enc_columns:
        X[tmp_col] = ordinal_encoder.fit_transform(
            X.loc[:, tmp_col].values.reshape(-1, 1)
        )
        
    #Dividing `PassengerId` with the largest value
    X['PassengerId'] = X['PassengerId']/X['PassengerId'].max()
    
    #Transforming `Cabin` to numerical data    
    cab_0 = X['Cabin'].str.split('/').str[0]
    cab_1 = X['Cabin'].str.split('/').str[1]
    cab_2 = X['Cabin'].str.split('/').str[2]
    
    cab_0 = cab_0.astype('str').map(change)
    cab_1 = cab_1.fillna(1)
    cab_1[cab_1.astype('int') == 0] = 0.5
    cab_1 = cab_1.astype('float')/cab_1.astype('float').max()
    cab_2 = cab_2.astype('str').map(change)

    X['Cabin'] = cab_0 * cab_1 * cab_2
    
    #Standard scaling selected columns
    std_scale_columns = [
        'Age', 'RoomService', 'FoodCourt', 
        'ShoppingMall', 'Spa', 'VRDeck',
        'Cabin'
    ]
    for tmp_col in std_scale_columns:
        X[tmp_col] = std_scaler.fit_transform(
            X.loc[:, tmp_col].values.reshape(-1, 1)
        )
        
    #Dropping `Name`, since I beleive it will not effect the prediction
    X = X.drop(['Name'], axis=1)
    
    return X

In [ ]:
X = data.iloc[:, :-1]
X

In [ ]:
X = preprocess_data(X)
X

# **Building the model and training**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
def score(y_test, y_pred):
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f'Precision => {precision}\nRecall => {recall}\nF1 score => {f1}')

**Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
score(y_test, y_pred)

**Ada Boost Classifier**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
score(y_test, y_pred)

**Gradient Boosting Classifier**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
score(y_test, y_pred)

**ANN**

In [ ]:
import tensorflow as tf

In [ ]:
X_train.shape

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation='relu', input_shape=(12, )),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3,
    restore_best_weights=True,
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    callbacks=[callback],
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
for key in history.history.keys():
    plt.plot(history.history[key])
plt.legend([key for key in history.history.keys()], loc='upper left')
plt.figure(figsize=(15, 15))
plt.show()

Using the validation set for training

In [ ]:
history_2 = model.fit(
    X_test, y_test,
    epochs=30,
    callbacks=[callback],
)

In [ ]:
test_data = pd.read_csv('../input/spaceship-titanic/test.csv')
sub_data = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

In [ ]:
test_data = preprocess_data(test_data)
test_data

In [ ]:
y_pred2 = model.predict(test_data)

In [ ]:
sub_data['Transported']=y_pred2
sub_data['Transported'] = sub_data['Transported'] > 0.5

In [ ]:
sub_data.head()

In [ ]:
sub_data.to_csv('submission.csv', index=False)